# Using DFTpy mixer

In [1]:
import qepy
import importlib

In [2]:
import numpy as np
from qepy.driver import Driver
from qepy.io import QEInput

In [3]:
from ase.io.trajectory import Trajectory
from ase.lattice.hexagonal import Graphene
from ase import Atoms
import matplotlib.pyplot as plt

In [4]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'"
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'nat': 1,
        'ntyp': 1,
        'occupations': "'smearing'"
    },
    '&electrons': {
        'conv_thr' : 1e-8
    },
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'atomic_species': ['Al  26.98 Al_ONCV_PBE-1.2.upf'],
    'k_points automatic': ['4 4 4 0 0 0'],
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
}

In [5]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.out')

In [6]:
for i in range(60):
    driver.diagonalize()
    driver.mix()
    converged = driver.check_convergence()
    print ('Iter: ',i,' - Conv: ', driver.get_scf_error())
    if converged : break
driver.calc_energy()

Iter:  0  - Conv:  0.07520824059877651
Iter:  1  - Conv:  0.0013949184647052578
Iter:  2  - Conv:  3.710927074024824e-05
Iter:  3  - Conv:  1.32989451870516e-07
Iter:  4  - Conv:  4.295314300787336e-10


-137.9144918110066

In [7]:
from dftpy.functional import Hartree

In [8]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.1.out')

In [9]:
rho = driver.get_density().copy()
coef = 0.7
for i in range(20):
    driver.diagonalize()
    #
    rho_new = driver.get_density().copy()
    drho = driver.data2field(rho_new-rho)
    error = Hartree.compute(drho).energy
    nc = np.abs(drho).sum()*driver.get_volume()/drho.size
    print ('Iter: ',i,' - Conv: ', error, 'dN:', nc)
    if error < 1e-8:
        driver.end_scf()
        break
    #
    rho = (1-coef)*rho + coef * rho_new
    driver.set_density(rho)
driver.calc_energy()

Iter:  0  - Conv:  0.03752851919933121 dN: 0.6417690036360949
Iter:  1  - Conv:  0.0007030693684957956 dN: 0.15211482969793375
Iter:  2  - Conv:  3.899234652126256e-05 dN: 0.045686645711029455
Iter:  3  - Conv:  3.359283387693953e-06 dN: 0.014066196101666017
Iter:  4  - Conv:  3.2333223714989856e-07 dN: 0.00441645212770983
Iter:  5  - Conv:  3.0518370843028964e-08 dN: 0.00136276412581075
Iter:  6  - Conv:  2.804704165066253e-09 dN: 0.00041966972947132445


-137.91449181104142

In [10]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.2.out')

In [11]:
rho_new = driver.get_density().copy()
coef = 0.7
hartree = driver.get_hartree()
xc = driver.get_exchange_correlation()
v_hxc = hartree[0] + xc[0]
for i in range(10):
    driver.diagonalize()
    #
    rho_new, rho = driver.get_density().copy(), rho_new
    drho = driver.data2field(rho_new-rho)
    error = Hartree.compute(drho).energy
    nc = np.abs(drho).sum()*driver.get_volume()/drho.size
    print ('Iter: ',i,' - Conv: ', error, 'dN:', nc)
    if error < 1e-8:
        driver.end_scf()
        break
    #
    hartree = driver.get_hartree()
    xc = driver.get_exchange_correlation()
    v_hxc_new = hartree[0] + xc[0]
    v_hxc = (1-coef)*v_hxc + coef*v_hxc_new
    extene = hartree[1] + xc[1]
    driver.set_external_potential(v_hxc, exttype=('hartree', 'xc'), extene=extene)
driver.get_energy()

Iter:  0  - Conv:  0.037437901915588066 dN: 0.640998305728986
Iter:  1  - Conv:  0.0013992599106399809 dN: 0.11937626385358018
Iter:  2  - Conv:  1.2556386014528264e-05 dN: 0.011465237457718627
Iter:  3  - Conv:  3.318016424378692e-07 dN: 0.0020696878129346325
Iter:  4  - Conv:  7.840132497960062e-09 dN: 0.00044031761906971776


-137.9144915709659

In [12]:
driver=Driver(qe_options=qe_options, iterative = True, logfile='tmp.3.out')

In [13]:
from dftpy.mixer import Mixer
driver.mixer = Mixer(scheme='pulay')

In [14]:
rho = driver.data2field(driver.get_density().copy())
for i in range(10):
    driver.diagonalize()
    #
    rho_new = driver.data2field(driver.get_density().copy())
    drho = rho_new - rho
    error = Hartree.compute(drho).energy
    nc = np.abs(drho).integral()
    print ('Iter: ',i,' - Conv: ', error, 'dN:', nc)
    if error < 1e-8:
        driver.end_scf()
        break
    #
    rho = driver.mixer(rho, rho_new, coef=0.7)
    density = driver.field2data(rho)
    driver.set_density(density)
driver.calc_energy()

Iter:  0  - Conv:  0.03755451077311118 dN: 0.6418754781189733
Iter:  1  - Conv:  0.0007007576828599942 dN: 0.15178804133319698
Iter:  2  - Conv:  1.3737606368086011e-05 dN: 0.028148121017143795
Iter:  3  - Conv:  8.15245894618318e-08 dN: 0.001362919446773032
Iter:  4  - Conv:  1.8093142856235335e-10 dN: 0.00012862004001710746


-137.9144918081112

In [15]:
ions = driver.get_dftpy_ions()

In [16]:
rho.write('tmp.xsf', ions=ions)